In [1]:
%matplotlib inline

In [2]:
import numpy as np
from nengo_spa import Vocabulary
from ssp.pointers import BaseVectors

# sample data
d = 32
rng = np.random.RandomState(seed=0)
vocab = Vocabulary(d, pointer_gen=BaseVectors(d, rng=rng))
vocab.populate('A')
A = vocab['A']

# nengo-spa implementation of binding matrix
T_valid = vocab.algebra.get_binding_matrix(A.v)

# equivalent method (transforming A with a fixed permutation matrix)
# note P is independent of A
I = np.eye(d)
P = np.vstack([np.roll(I, shift=i, axis=0) for i in range(d)])
T = P.dot(A.v).reshape(d, d).T

assert np.allclose(T, T_valid)

In [3]:
from scipy.linalg import logm

def get_T(P):
    return vocab.algebra.get_binding_matrix(P.v)

def get_lnT(P, exp=1):
    # exp doesnt matter
    # needs explanation
    # reminder: ZOH and Intro to Stochastic Processes (Greogry F. Lawler)
    P = (P ** exp)
    T = get_T(P)
    return logm(T) / exp

In [4]:
vocab.populate('X')
X = vocab['X']

In [5]:
assert np.allclose(get_lnT(X, 0.1), get_lnT(X, 0.2))
assert np.allclose(get_lnT(X, 0.01), get_lnT(X, 1))
# ^ because ln(X^exp)/exp = ln(X)
#   and because get_binding_matrix is linear
assert np.allclose(
    get_T(4.20 * X),
    4.20 * get_T(X),
)

In [6]:
dx_by_dt = 0.1
dt = 0.5
dx = dx_by_dt * dt  # :+1: engineering

# can also just pass in X if you play with substitution of variables
lnT = get_lnT(X ** dx_by_dt)

# Euler's method to simulate continuous system:
# \dot{B} = (lnT).dot(B)
# => B[t+euler_dt] = euler_dt*(lnT).dot(B[t]) + B[t]
#    where euler_dt is sufficiently small
# starting from B[0] = A, simulating until B[dt]

euler_dt = 1e-6
steps = int(dt / euler_dt)
print(steps)
B = A.v
for step in range(steps):
    B = euler_dt*(lnT.dot(B)) + B

500000


In [7]:
assert np.allclose(B, (A * X**dx).v, atol=1e-7)

In [8]:
vocab.populate('Y')
Y = vocab['Y']

In [9]:
dx_by_dt = 0.4
dy_by_dt = 0.2

dt = 0.9
dx = dx_by_dt * dt  # :+1: engineering
dy = dy_by_dt * dt  # :+1: engineering

# can also just pass in X if you play with substitution of variables
lnT = get_lnT(X ** dx_by_dt * Y ** dy_by_dt)

# Euler's method to simulate continuous system:
# \dot{B} = (lnT).dot(B)
# => B[t+euler_dt] = euler_dt*(lnT).dot(B[t]) + B[t]
#    where euler_dt is sufficiently small
# starting from B[0] = A, simulating until B[dt]

euler_dt = 1e-6
steps = int(dt / euler_dt)
print(steps)
B = A.v
for step in range(steps):
    B = euler_dt*(lnT.dot(B)) + B

900000


In [10]:
assert np.allclose(B, (A * X**dx * Y**dy).v, atol=1e-7)

In [11]:
# again get_binding_matrix is linear
assert np.allclose(
    get_T(X + Y),
    get_T(X) + get_T(Y),
)
# and ln(X^x Y^y) = xln(X) + yln(Y)
# so we get the first big result:
assert np.allclose(
    lnT,
    dx_by_dt * get_lnT(X) +  dy_by_dt * get_lnT(Y)
)

In [12]:
from scipy.linalg import eig, inv
s, v = eig(get_T(X))
assert np.allclose(
    get_lnT(X),
    v.dot(np.eye(d)*np.log(s)).dot(inv(v)).real,
)

In [13]:
assert np.allclose(np.abs(s), 1)
assert np.allclose(np.log(s).real, 0)
assert np.allclose(
    np.sort(np.log(s).imag),
    np.sort(np.log(np.fft.fft(X.v)).imag)
)

In [14]:
def unitary_lnT(T):
    # you can move the ln inside of T()
    return vocab.algebra.get_binding_matrix(
        np.fft.ifft(np.log(np.fft.fft(T.v))).real
    )

assert np.allclose(
    unitary_lnT(X),
    get_lnT(X),
)

assert np.allclose(
    lnT,
    dx_by_dt * unitary_lnT(X) + dy_by_dt * unitary_lnT(Y)
)

In [15]:
# and so we get the final (?) result
# expressing it as a binding again

import nengo_spa as spa
lnX = spa.SemanticPointer(np.fft.ifft(np.log(np.fft.fft(X.v))).real)
lnY = spa.SemanticPointer(np.fft.ifft(np.log(np.fft.fft(Y.v))).real)
dXY = dx_by_dt * lnX + dy_by_dt * lnY

assert np.allclose(
    get_T(dXY),
    lnT,
)

euler_dt = 1e-6
steps = int(dt / euler_dt)
print(steps)
B = A
for step in range(steps):
    B = euler_dt*( dXY * B ) + B
    B.name = ""
B = B.v

900000


In [16]:
assert np.allclose(B, (A * X**dx * Y**dy).v, atol=1e-7)